In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['abcnews-date-text.csv']


In [3]:
data = pd.read_csv("../input/abcnews-date-text.csv", error_bad_lines=False)

In [4]:
data_text = data[:300000][['headline_text']]
data_text['index']=data_text.index
documents = data_text

In [5]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer , SnowballStemmer
from nltk.stem.porter import *
np.random.seed(400)

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
            result.append(lemmatize_stemming(token))
    return result

In [28]:
doc_num = 4310
doc_sample = documents[documents['index']==doc_num].values[0][0]

print("original document")
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))
    

original document
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [16]:
doc_sample

'rain helps dampen bushfires'

In [23]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [29]:
processed_docs = documents['headline_text'].map(preprocess)

In [30]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [31]:
dictionary= gensim.corpora.Dictionary(processed_docs)

In [37]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count +=1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [38]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [39]:
bow_corpus= [dictionary.doc2bow(doc) for doc in processed_docs]

In [40]:
bow_corpus[doc_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [41]:
bow_doc_4310 = bow_corpus[doc_num]
for i in range(len(bow_doc_4310)):
    print ("Word{} (\"{}\")) appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word71 ("bushfir")) appears 1 time.
Word107 ("help")) appears 1 time.
Word462 ("rain")) appears 1 time.
Word3530 ("dampen")) appears 1 time.


In [42]:
from gensim import corpora,models
tfidf = models.TfidfModel(bow_corpus)

In [43]:
corpus_tfidf = tfidf[bow_corpus]


In [44]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


In [45]:
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [46]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.024*"open" + 0.021*"test" + 0.018*"world" + 0.017*"win" + 0.014*"lead" + 0.014*"south" + 0.012*"take" + 0.012*"timor" + 0.011*"strike" + 0.010*"east"


Topic: 1 
Words: 0.034*"report" + 0.030*"help" + 0.017*"urg" + 0.017*"deal" + 0.015*"blaze" + 0.015*"inquiri" + 0.012*"firefight" + 0.012*"close" + 0.011*"bushfir" + 0.011*"resid"


Topic: 2 
Words: 0.038*"crash" + 0.022*"closer" + 0.017*"die" + 0.016*"road" + 0.016*"coast" + 0.016*"sydney" + 0.014*"train" + 0.013*"dead" + 0.012*"kill" + 0.012*"gold"


Topic: 3 
Words: 0.041*"plan" + 0.035*"council" + 0.031*"govt" + 0.030*"water" + 0.017*"urg" + 0.016*"group" + 0.012*"fund" + 0.012*"chang" + 0.012*"concern" + 0.012*"consid"


Topic: 4 
Words: 0.023*"hospit" + 0.022*"labor" + 0.019*"defend" + 0.019*"elect" + 0.016*"protest" + 0.016*"minist" + 0.015*"power" + 0.014*"govt" + 0.014*"work" + 0.013*"begin"


Topic: 5 
Words: 0.045*"warn" + 0.020*"fight" + 0.017*"england" + 0.017*"nuclear" + 0.016*"year" + 0.014*"action" + 0

In [47]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [48]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.011*"market" + 0.009*"price" + 0.007*"black" + 0.006*"profit" + 0.006*"record" + 0.005*"resign" + 0.005*"time" + 0.005*"eagl" + 0.005*"share" + 0.005*"wait"


Topic: 1 Word: 0.009*"plan" + 0.008*"govt" + 0.008*"council" + 0.007*"farm" + 0.007*"drought" + 0.007*"farmer" + 0.006*"telstra" + 0.006*"urg" + 0.006*"wind" + 0.006*"group"


Topic: 2 Word: 0.008*"liber" + 0.006*"beazley" + 0.006*"quit" + 0.005*"interview" + 0.005*"surgeri" + 0.005*"polic" + 0.005*"coach" + 0.005*"maintain" + 0.005*"militari" + 0.004*"warrior"


Topic: 3 Word: 0.012*"nuclear" + 0.006*"bushfir" + 0.006*"iran" + 0.006*"celebr" + 0.005*"korea" + 0.005*"flag" + 0.004*"lake" + 0.004*"evid" + 0.004*"dump" + 0.004*"villag"


Topic: 4 Word: 0.011*"govt" + 0.010*"water" + 0.009*"health" + 0.009*"fund" + 0.008*"urg" + 0.008*"plan" + 0.007*"indigen" + 0.007*"council" + 0.006*"boost" + 0.006*"hous"


Topic: 5 Word: 0.023*"polic" + 0.018*"charg" + 0.016*"crash" + 0.013*"court" + 0.013*"murder" + 0.012*"inves

In [49]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [51]:
for index, score in sorted(lda_model[bow_corpus[doc_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6199342608451843	 
Topic: 0.034*"report" + 0.030*"help" + 0.017*"urg" + 0.017*"deal" + 0.015*"blaze" + 0.015*"inquiri" + 0.012*"firefight" + 0.012*"close" + 0.011*"bushfir" + 0.011*"resid"

Score: 0.22005116939544678	 
Topic: 0.019*"school" + 0.018*"drought" + 0.018*"farmer" + 0.015*"fund" + 0.014*"price" + 0.014*"market" + 0.013*"rise" + 0.013*"boost" + 0.012*"rain" + 0.012*"feder"

Score: 0.020001819357275963	 
Topic: 0.024*"open" + 0.021*"test" + 0.018*"world" + 0.017*"win" + 0.014*"lead" + 0.014*"south" + 0.012*"take" + 0.012*"timor" + 0.011*"strike" + 0.010*"east"

Score: 0.020001819357275963	 
Topic: 0.038*"crash" + 0.022*"closer" + 0.017*"die" + 0.016*"road" + 0.016*"coast" + 0.016*"sydney" + 0.014*"train" + 0.013*"dead" + 0.012*"kill" + 0.012*"gold"

Score: 0.020001819357275963	 
Topic: 0.041*"plan" + 0.035*"council" + 0.031*"govt" + 0.030*"water" + 0.017*"urg" + 0.016*"group" + 0.012*"fund" + 0.012*"chang" + 0.012*"concern" + 0.012*"consid"

Score: 0.0200018193572759

In [52]:
for index, score in sorted(lda_model_tfidf[bow_corpus[doc_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5791903734207153	 
Topic: 0.009*"water" + 0.007*"restrict" + 0.006*"titl" + 0.006*"uranium" + 0.006*"eas" + 0.006*"break" + 0.006*"hill" + 0.006*"kangaroo" + 0.005*"rate" + 0.005*"open"

Score: 0.26076966524124146	 
Topic: 0.012*"nuclear" + 0.006*"bushfir" + 0.006*"iran" + 0.006*"celebr" + 0.005*"korea" + 0.005*"flag" + 0.004*"lake" + 0.004*"evid" + 0.004*"dump" + 0.004*"villag"

Score: 0.02000747248530388	 
Topic: 0.008*"hick" + 0.008*"firefight" + 0.008*"east" + 0.008*"blaze" + 0.007*"export" + 0.007*"south" + 0.007*"bird" + 0.006*"solomon" + 0.006*"cyclon" + 0.006*"crew"

Score: 0.020006336271762848	 
Topic: 0.009*"plan" + 0.008*"govt" + 0.008*"council" + 0.007*"farm" + 0.007*"drought" + 0.007*"farmer" + 0.006*"telstra" + 0.006*"urg" + 0.006*"wind" + 0.006*"group"

Score: 0.02000594697892666	 
Topic: 0.011*"govt" + 0.010*"water" + 0.009*"health" + 0.009*"fund" + 0.008*"urg" + 0.008*"plan" + 0.007*"indigen" + 0.007*"council" + 0.006*"boost" + 0.006*"hous"

Score: 0.02000536

In [53]:
unseen_document = "My favorite sports activities are running and swimming."

In [54]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4200017750263214	 Topic: 0.024*"open" + 0.021*"test" + 0.018*"world" + 0.017*"win" + 0.014*"lead"
Score: 0.2225508838891983	 Topic: 0.023*"hospit" + 0.022*"labor" + 0.019*"defend" + 0.019*"elect" + 0.016*"protest"
Score: 0.21739985048770905	 Topic: 0.028*"iraq" + 0.018*"talk" + 0.016*"australia" + 0.015*"troop" + 0.012*"storm"
Score: 0.020009607076644897	 Topic: 0.045*"warn" + 0.020*"fight" + 0.017*"england" + 0.017*"nuclear" + 0.016*"year"
Score: 0.020008262246847153	 Topic: 0.041*"plan" + 0.035*"council" + 0.031*"govt" + 0.030*"water" + 0.017*"urg"
Score: 0.020007528364658356	 Topic: 0.019*"school" + 0.018*"drought" + 0.018*"farmer" + 0.015*"fund" + 0.014*"price"
Score: 0.02000551111996174	 Topic: 0.034*"report" + 0.030*"help" + 0.017*"urg" + 0.017*"deal" + 0.015*"blaze"
Score: 0.02000551111996174	 Topic: 0.038*"crash" + 0.022*"closer" + 0.017*"die" + 0.016*"road" + 0.016*"coast"
Score: 0.02000551111996174	 Topic: 0.074*"polic" + 0.031*"charg" + 0.027*"court" + 0.023*"face" 